In [1]:
!pip install max --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 MB 4.5 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 4059, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 4059 (delta 151), reused 127 (delta 127), pack-reused 3893 (from 2)
Receiving objects: 100% (4059/4059), 94.69 MiB | 27.21 MiB/s, done.
Resolving deltas: 100% (2489/2489), done.


In [3]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.7.12 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [4]:
import max.support.notebook

In [5]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [6]:
mojo_code = """
from memory import UnsafePointer
from gpu import thread_idx, block_dim, block_idx
from gpu.host import DeviceContext
from testing import assert_equal

# ANCHOR: add_10_2d
alias SIZE = 2
alias BLOCKS_PER_GRID = 1
alias THREADS_PER_BLOCK = (3, 3)
alias dtype = DType.float32


fn add_10_2d(
    output: UnsafePointer[Scalar[dtype]],
    a: UnsafePointer[Scalar[dtype]],
    size: Int,
):
    row = thread_idx.y
    col = thread_idx.x
    # FILL ME IN (roughly 2 lines)
    if row < size and col < size:
      output[row*size+col] = a[row*size+col] + 10


# ANCHOR_END: add_10_2d


def main():
    with DeviceContext() as ctx:
        out = ctx.enqueue_create_buffer[dtype](SIZE * SIZE).enqueue_fill(0)
        expected = ctx.enqueue_create_host_buffer[dtype](
            SIZE * SIZE
        ).enqueue_fill(0)
        a = ctx.enqueue_create_buffer[dtype](SIZE * SIZE).enqueue_fill(0)
        with a.map_to_host() as a_host:
            # row-major
            for i in range(SIZE):
                for j in range(SIZE):
                    a_host[i * SIZE + j] = i * SIZE + j
                    expected[i * SIZE + j] = a_host[i * SIZE + j] + 10

        ctx.enqueue_function[add_10_2d](
            out.unsafe_ptr(),
            a.unsafe_ptr(),
            SIZE,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        ctx.synchronize()

        with out.map_to_host() as out_host:
            print("out:", out_host)
            print("expected:", expected)
            for i in range(SIZE):
                for j in range(SIZE):
                    assert_equal(out_host[i * SIZE + j], expected[i * SIZE + j])

"""

In [7]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p04/p04.mojo")

In [8]:
!cd /content/mojo-gpu-puzzles && uv run poe p04

Using CPython 3.11.13 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
pastel               ------------------------------     0 B/5.82 KiB
⠙ Preparing packages... (0/7)
pastel               ------------------------------     0 B/5.82 KiB
⠙ Preparing packages... (0/7)
pastel               ------------------------------     0 B/5.82 KiB
⠙ Preparing packages... (0/7)
pastel               ------------------------------     0 B/5.82 KiB
pyyaml               ------------------------------     0 B/745.07 KiB
⠙ Preparing packages... (0/7)
pastel               ------------------------------     0 B/5.82 KiB
pyyaml               ------------------------------ 14.88 KiB/745.07 KiB
⠙ Preparing packages... (0/7)
pastel               ------------------------------ 5.82 KiB/5.82 KiB
pyyaml               ------------------------------ 14.88 KiB/745.07 KiB
⠙ Preparing packages... (0/7)
pyya